In [ ]:
from bokeh.plotting import figure, output_notebook, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, CrosshairTool, WheelZoomTool
from bokeh.models import ColorBar, ResetTool, PanTool, LinearColorMapper

import bokeh.palettes as palettes

import matplotlib as mpl
output_notebook()

def get_structures(mol_df, imgformat='png'):
    
    if imgformat not in ('png', 'svg'):
        raise ValueError('imgformat can only be "svg" or "png"')
    
    if imgformat == 'png':
        img_path = []
        for mol in range(len(mol_df)):
            Draw.MolToFile(mol_df.ROMol[mol],
                        "static/imgs/"+mol_df.name[mol]+".png",
                        imageType="png",
                        fitImage=True,
                        size=(200, 200))

            #mol_df["img_path"][mol] = "static/imgs/"+mol_df.name[mol]+".svg"
            img_path.append("static/imgs/"+mol_df.name[mol]+".png")

        mol_df["img_path"] = img_path
        return mol_df
    
    elif imgformat == 'svg':
        
        img_path = []
        for mol in range(len(mol_df)):
            Draw.MolToFile(mol_df.ROMol[mol],
                        "static/imgs/"+mol_df.name[mol]+".svg",
                        imageType="svg",
                        fitImage=True,
                        size=(200, 200))

            #mol_df["img_path"][mol] = "static/imgs/"+mol_df.name[mol]+".svg"
            img_path.append("static/imgs/"+mol_df.name[mol]+".svg")

        mol_df["img_path"] = img_path
        return mol_df


def interactiveplot(df):
    source = ColumnDataSource(df)
    source.data['HCPSA'] = source.data['HCPSA']/6

    color_maper = LinearColorMapper(palette='Viridis256', low=min(mols_pr.activity), high=max(mols_pr.activity))
    #preparing the hover tooltip with HTML template
    
    hover = HoverTool(tooltips = """
                <div>
                    <img src="@img_path" width="200" height="200"></img>
                </div> 
                <div>
                    <span style="font-size: 14px; font-weight: bold;">@name</span>
                </div>
                <br>
                <div>
                    <span style="font-size: 12px; font-weight: bold;">x,y:</span>
                    <span style="font-size: 12px;">@x, @y</span>
                </div>
                
                <div>
                    <span style="font-size: 12px; font-weight: bold;">HCPSA:</span>
                    <span style="font-size: 12px;">@HCPSA</span>
                </div>
                <div>
                    <span style="font-size: 12px; font-weight: bold;">Activity:</span>
                    <span style="font-size: 12px;">@activity</span>
                </div>
                    """
                     )

    TOOLS=[hover, CrosshairTool(), WheelZoomTool(), ResetTool(), PanTool()]

    p = figure(plot_width=800, plot_height=800,title="Mouse over the dots", tools=TOOLS, toolbar_location="above")

    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None

    p.scatter('x','y',size='HCPSA',alpha = 0.7,color='activity_color',source = source)

    color_bar = ColorBar(color_mapper=color_maper, label_standoff=12,
                         border_line_color=None, location=(0,0))

    p.add_layout(color_bar, 'right')
    

    output_file("pca_scatter.html")

    show(p)